In [ ]:
!rm customers.sqlite food_new.csv

In [34]:
# Dependencies
import pandas as pd
import numpy as np
import os

# Data Cleaning

In [35]:
# Path of the CSV file
csvfile = "restaurant_updated.csv"

In [36]:
# Read CSV file into a pandas DataFrame
df = pd.read_csv(csvfile, dtype=object)

In [37]:
# Preview DataFrame
# Note that some rows are missing gender
df.head()

,id,the_geom,city,latitude,longitude,invalid_the_geom,storenumber,storetype,address,cartodb_id,...,Population_2014,FF_per_capita_2009,FF_per_capita_2014,p_change_FF_Per_Capita,Grocery_per_capita_2009,Grocery_per_capita_2014,p_change_in_grocery_per_capita,p_change_in_Access,p_Change_in_Obesity,p_change_in_income
0,1,0101000020E6100000541C075E2DEB60C09F909DB7B187...,SITKA,57.060111,-135.349288,Point,7972,FREESTANDING,913 HALIBUT POINT RD,14168,...,8883,0.0004504,0.00045,-0.02,0.0004504,0.0004503,-0.02,0,-3.37,18.06
1,2,0101000020E610000056F31C916FD260C0C8EC2C7AA72E...,JUNEAU,58.364486,-134.576119,Point,6123,FREESTANDING,2285 TROUT ST,14169,...,32511,0.000703437,0.00095,35.55,0.000223821,0.00018455,-17.54,0,9.16,18.68
2,3,0101000020E61000009E42AED4B3425DC0F9156BB8C835...,CLARKSTON,46.420188,-117.042226,Point,10610,FREESTANDING,350 BRIDGE ST,8632,...,22177,0.000416223,0.00045,8.34,0.000231235,0.00022546,-2.5,0,14.38,18.39
3,4,0101000020E610000032022A1C41D25DC0B7ED7BD45F21...,RICHLAND,46.260737,-119.285224,Point,6556,FREESTANDING,101 D WELLSIAN WAY,13945,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03
4,5,0101000020E6100000624B8FA67AF25DC0A48CB800341C...,PROSSER,46.220337,-119.788736,Point,11889,FREESTANDING,103 MERLOT DR,13886,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03


In [38]:
df = df.drop(['address', 'phone'], 1)

In [39]:
df.head()

,id,the_geom,city,latitude,longitude,invalid_the_geom,storenumber,storetype,cartodb_id,state,...,Population_2014,FF_per_capita_2009,FF_per_capita_2014,p_change_FF_Per_Capita,Grocery_per_capita_2009,Grocery_per_capita_2014,p_change_in_grocery_per_capita,p_change_in_Access,p_Change_in_Obesity,p_change_in_income
0,1,0101000020E6100000541C075E2DEB60C09F909DB7B187...,SITKA,57.060111,-135.349288,Point,7972,FREESTANDING,14168,AK,...,8883,0.0004504,0.00045,-0.02,0.0004504,0.0004503,-0.02,0,-3.37,18.06
1,2,0101000020E610000056F31C916FD260C0C8EC2C7AA72E...,JUNEAU,58.364486,-134.576119,Point,6123,FREESTANDING,14169,AK,...,32511,0.000703437,0.00095,35.55,0.000223821,0.00018455,-17.54,0,9.16,18.68
2,3,0101000020E61000009E42AED4B3425DC0F9156BB8C835...,CLARKSTON,46.420188,-117.042226,Point,10610,FREESTANDING,8632,WA,...,22177,0.000416223,0.00045,8.34,0.000231235,0.00022546,-2.5,0,14.38,18.39
3,4,0101000020E610000032022A1C41D25DC0B7ED7BD45F21...,RICHLAND,46.260737,-119.285224,Point,6556,FREESTANDING,13945,WA,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03
4,5,0101000020E6100000624B8FA67AF25DC0A48CB800341C...,PROSSER,46.220337,-119.788736,Point,11889,FREESTANDING,13886,WA,...,243991,0.00061896,0.00066,5.95,0.000135539,0.00012296,-9.28,-20,7.5,15.03


In [40]:
new_csv = "cleaned.csv"
df.to_csv(new_csv, index=False)

# Database Creation

In [41]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [42]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///foodstore_seven.sqlite")

In [43]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [47]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'food_seven'
    
    id=Column(Integer,primary_key=True)
    the_geom=Column(Text)
    city=Column(Text)
    latitude=Column(Float)
    longitude=Column(Float)
    invalid_the_geom=Column(Text)
    storenumber=Column(Float)
    storetype=Column(String)
    #address=Column(String)
    cartodb_id=Column(Float)
    state=Column(Text)
    zip=Column(Float)
    #phone=Column(Float)
    playplace=Column(Text)
    drivethru=Column(Text)
    archcard=Column(Text)
    freewifi=Column(Text)
    storeurl=Column(Text)
    cartodb_georef_status=Column(Float)
    COUNTY=Column(Text)
    Grocery_2009=Column(Float)
    Convenience_2009=Column(Float)
    Fast_Food_Restaurants_2009=Column(Float)
    Adult_Diabetes_p_2009=Column(Float)
    Adult_Obsesity_p_2009=Column(Float)
    Median_Household_Income_2009=Column(Float)
    Limited_Access_Populationp_2009=Column(Float)
    Limited_Access_Population_2009=Column(Float)
    Population_2009=Column(Float)
    Grocery_2014=Column(Float)
    Convenience_2014=Column(Float)
    Fast_Food_Restaurants_2014=Column(Float)
    Adult_Diabetes_2014=Column(Float)
    Adult_Obesity_2014=Column(Float)
    Median_Household_Income_2014=Column(Float)
    Limited_Access_p_2014=Column(Float)
    Limited_Access_2014=Column(Float)
    Population_2014=Column(Float)
    FF_per_capita_2009=Column(Float)
    FF_per_capita_2014=Column(Float)
    p_change_FF_Per_Capita=Column(Float)
    Grocery_per_capita_2009=Column(Float)
    Grocery_per_capita_2014=Column(Float)
    p_change_in_grocery_per_capita=Column(Float)
    p_change_in_Access=Column(Float)
    p_Change_in_Obesity=Column(Float)
    p_change_in_income=Column(Float)
    
  

In [48]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [49]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(new_csv)

In [50]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [51]:
# Data is just a list of dictionaries that represent each row of data
print(data[:5])

#data[:1]

[{'id': 1, 'the_geom': '0101000020E6100000541C075E2DEB60C09F909DB7B1874C40', 'city': 'SITKA', 'latitude': 57.060111, 'longitude': -135.349288, 'invalid_the_geom': 'Point', 'storenumber': 7972, 'storetype': 'FREESTANDING', 'cartodb_id': 14168, 'state': 'AK', 'zip': 99835, 'playplace': False, 'drivethru': True, 'archcard': True, 'freewifi': True, 'storeurl': 'http://www.mcalaska.com/7972', 'cartodb_georef_status': True, 'COUNTY': 'Sitka', 'Grocery_2009': 4, 'Convenience_2009': 2, 'Fast_Food_Restaurants_2009': 4, 'Adult_Diabetes_p_2009': 7.0, 'Adult_Obsesity_p_2009': 29.7, 'Median_Household_Income_2009': 58000, 'Limited_Access_Populationp_2009': 18, 'Limited_Access_Population_2009': 1617, 'Population_2009': 8881, 'Grocery_2014': 4, 'Convenience_2014': 2, 'Fast_Food_Restaurants_2014': 4, 'Adult_Diabetes_2014': 6.7, 'Adult_Obesity_2014': 28.7, 'Median_Household_Income_2014': 68472, 'Limited_Access_p_2014': 18, 'Limited_Access_2014': 1632, 'Population_2014': 8883, 'FF_per_capita_2009': 0.000

In [52]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [53]:
# Save the reference to the `customers` table as a variable called `food`
table = sqlalchemy.Table('food_seven', metadata, autoload=True)

In [54]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [55]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [56]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from food_seven limit 5").fetchall()

[(1, '0101000020E6100000541C075E2DEB60C09F909DB7B1874C40', 'SITKA', 57.060111, -135.349288, 'Point', 7972.0, 'FREESTANDING', 14168.0, 'AK', 99835.0, None, '0', '1', '1', '1', 'http://www.mcalaska.com/7972', 1.0, 'Sitka', 4.0, 2.0, 4.0, 7.0, 29.7, 58000.0, 18.0, 1617.0, 8881.0, 4.0, 2.0, 4.0, 6.7, 28.7, 68472.0, 18.0, 1632.0, 8883.0, 0.00045039999999999994, 0.00045, -0.02, 0.00045039999999999994, 0.0004503, -0.02, 0.0, -3.37, 18.06),
 (2, '0101000020E610000056F31C916FD260C0C8EC2C7AA72E4D40', 'JUNEAU', 58.364486, -134.576119, 'Point', 6123.0, 'FREESTANDING', 14169.0, 'AK', 99801.0, None, '1', '1', '1', '1', 'http://www.mcalaska.com/6123', 1.0, 'Juneau', 7.0, 4.0, 22.0, 5.5, 26.2, 69844.0, 32.0, 10138.0, 31275.0, 6.0, 9.0, 31.0, 5.9, 28.6, 82892.0, 32.0, 10063.0, 32511.0, 0.000703437, 0.00095, 35.55, 0.000223821, 0.00018455, -17.54, 0.0, 9.16, 18.68),
 (3, '0101000020E61000009E42AED4B3425DC0F9156BB8C8354740', 'CLARKSTON', 46.420188, -117.042226, 'Point', 10610.0, 'FREESTANDING', 8632.0, '